In [14]:
from google.cloud import bigquery
client = bigquery.Client(project="umt-msba")
import pandas as pd

## Table 1

In [19]:
query = """
 SELECT 
    EXTRACT(DATE FROM datetime) AS sale_date,  -- Extract the sale date
    EXTRACT(HOUR FROM datetime) AS sale_hour,  -- Extract the sale hour
    ROUND(SUM(total), 2) AS total_sales,       -- Calculate total sales
    COUNT(DISTINCT CONCAT(
        CAST(EXTRACT(DATE FROM datetime) AS STRING),
        CAST(register_no AS STRING),
        CAST(emp_no AS STRING),
        CAST(trans_no AS STRING)
    )) AS num_transactions,                    -- Count distinct transactions
    SUM(
        CASE
            WHEN trans_status IN ('V', 'R') THEN -1
            ELSE 1
        END
    ) AS total_items                           -- Calculate total items accounting for returns/voids
FROM `umt-msba.wedge_transactions.transArchive*` AS trans
JOIN `umt-msba.wedge_transactions.department_lookup` AS depts
    ON trans.department = depts.department
WHERE trans.department NOT IN (0, 15)           -- Exclude departments 0 and 15
  AND card_no != 3                              -- Exclude non-owner transactions
  AND (trans_status IS NULL
      OR trans_status IN ('V', 'R', '', ' '))   -- Include only the specified trans_status values
GROUP BY sale_date, sale_hour                   -- Group by date and hour
ORDER BY sale_date, sale_hour;                  -- Order by date and hour
"""

query_job_1 = client.query(query)

df_1 = query_job_1.to_dataframe()

print(df_1.head())


    sale_date  sale_hour  total_sales  num_transactions  total_items
0  2010-01-01          9       717.50                18          175
1  2010-01-01         10      2665.77                55          757
2  2010-01-01         11      2824.45                65          778
3  2010-01-01         12      2523.37                75          773
4  2010-01-01         13      3789.56               104         1091


## Table 2

In [20]:
query = """
     SELECT 
    card_no,                                  -- Owner's card number
    EXTRACT(YEAR FROM datetime) AS sale_year, -- Extract the sale year
    EXTRACT(MONTH FROM datetime) AS sale_month, -- Extract the sale month
    ROUND(SUM(total), 2) AS total_sales,      -- Calculate total sales
    COUNT(DISTINCT CONCAT(
        CAST(EXTRACT(DATE FROM datetime) AS STRING),
        CAST(register_no AS STRING),
        CAST(emp_no AS STRING),
        CAST(trans_no AS STRING)
    )) AS num_transactions,                    -- Count distinct transactions
    SUM(
        CASE
            WHEN trans_status IN ('V', 'R') THEN -1
            ELSE 1
        END
    ) AS total_items                           -- Calculate total items accounting for returns/voids
FROM `umt-msba.wedge_transactions.transArchive*` AS trans
JOIN `umt-msba.wedge_transactions.department_lookup` AS depts
    ON trans.department = depts.department
WHERE trans.department NOT IN (0, 15)          -- Exclude departments 0 and 15
  AND card_no != 3                             -- Exclude non-owner transactions
  AND (trans_status IS NULL
      OR trans_status IN ('V', 'R', '', ' '))  -- Include only the specified trans_status values
GROUP BY card_no, sale_year, sale_month        -- Group by owner, year, and month
ORDER BY card_no, sale_year, sale_month;       -- Order by owner, year, and month
"""

query_job_2 = client.query(query)

df_2 = query_job_2.to_dataframe()

print(df_2.head())


   card_no  sale_year  sale_month  total_sales  num_transactions  total_items
0  10000.0       2010          10        65.87                 4           21
1  10000.0       2010          11        53.12                 2           20
2  10000.0       2010          12        17.34                 1            6
3  10000.0       2011           1        60.40                 4           23
4  10000.0       2011           2        19.65                 1            4


## Table 3

In [21]:
query = """
     SELECT DISTINCT upc
    , description
    , trans.department AS `department number`
    , depts.dept_name AS `department name`
    , EXTRACT(YEAR FROM datetime) AS year
    , EXTRACT(MONTH FROM datetime) AS month
    , ROUND(SUM(total), 2) as sales
    , COUNT(DISTINCT CONCAT(
        CAST(EXTRACT(DATE FROM datetime) AS STRING),
        CAST(register_no AS STRING),
        CAST(emp_no AS STRING),
        CAST(trans_no AS STRING)
        )) AS transactions
    ,SUM(
        CASE
        WHEN trans_status IN ('V', 'R') THEN -1
        ELSE 1
    END
    ) AS items
 
    FROM `umt-msba.wedge_transactions.transArchive*` as trans
    JOIN `umt-msba.wedge_transactions.department_lookup` as depts
    ON trans.department = depts.department
    WHERE trans.department NOT IN (0, 15)
    AND card_no != 3
    AND (trans_status IS NULL
    OR trans_status IN ('V', 'R', '', ' '))
    GROUP BY upc, description, `department number`, `department name`, year, month
    ORDER BY upc, description, `department number`, `department name`, year, month;
"""

query_job_3 = client.query(query)

df_3 = query_job_3.to_dataframe()

print(df_3.head())


  upc              description  department number   department name  year  \
0   0              BULK Coupon                3.0              BULK  2010   
1   0              BULK Coupon                3.0              BULK  2010   
2   0            FROZEN Coupon                6.0            FROZEN  2010   
3   0  PACKAGED GROCERY Coupon                1.0  PACKAGED GROCERY  2010   
4   0  PACKAGED GROCERY Coupon                1.0  PACKAGED GROCERY  2010   

   month  sales  transactions  items  
0      1   2.00             1     -1  
1      2   1.00             1     -1  
2      2   2.00             1     -1  
3      1  14.48             2     -4  
4      2  15.49             5     -5  


In [23]:
import sqlite3

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('sales_summary.db')

# 1. Insert 'Sales by Date by Hour' DataFrame into the database
df_1.to_sql('sales_by_date_by_hour', conn, if_exists='replace', index=False)
print("Inserted 'Sales by Date by Hour' table successfully.")

# 2. Insert 'Sales by Owner by Year by Month' DataFrame into the database
df_2.to_sql('sales_by_owner_by_year_by_month', conn, if_exists='replace', index=False)
print("Inserted 'Sales by Owner by Year by Month' table successfully.")

# 3. Insert 'Sales by Product Description by Year by Month' DataFrame into the database
df_3.to_sql('sales_by_product_description_by_year_by_month', conn, if_exists='replace', index=False)
print("Inserted 'Sales by Product Description by Year by Month' table successfully.")

# Commit and close the connection
conn.commit()
conn.close()


Inserted 'Sales by Date by Hour' table successfully.
Inserted 'Sales by Owner by Year by Month' table successfully.
Inserted 'Sales by Product Description by Year by Month' table successfully.
